#### Cargar Pandas

In [1]:
import pandas as pd

##### Cargar los dataframes que contienen las variaables para luego juntarlas  en un solo dataframe 

In [2]:
df_Escolar=pd.read_csv("Data_Procesada\\Escolar.csv",sep=",")
df_Pobreza=pd.read_csv("Data_Procesada\\Pobreza.csv",sep=",")
df_Homicidios=pd.read_csv("Data_Procesada\\Tasa_Homicidios.csv",sep=",")

####  Aplicar tranformaciones al dataframe de datos escolares para poder luego hacer la fusion con los datos de homicidios y pobreza 

In [ ]:
## Agrupar el dataframe de datos de educacion por Año y por Provincia 

data_Escolar=df_Escolar.groupby(["Año","Provincia"],as_index=False).agg({
    "Total_Estudiantes":"sum",
    "Promovidos":"sum",
    "No_Promovidos" : "sum",
    "Abandono":"sum"
    })

## Ordenar dataframe por año 
data_Escolar.sort_values(["Año"])


#Renombrar las columnas de los datos de educacion para darle un identificador claro cuando se estructure el dataframe final con las demas variables 

data_Escolar=data_Escolar.rename(columns={
    "Promovidos":"Estudiantes_Promovidos",
    "No_Promovidos":"Estudiantes_No_Promovidos",
    "Abandono":"Estudiantes_Abandono"
})
data_Escolar


## Calcular tasa de desercion escolar 

def tasa_abandono(Estudiantes_Abandono,Total_Estudiantes):
    if Estudiantes_Abandono==0 :
        return 0
    tasa= (Estudiantes_Abandono/Total_Estudiantes)*100
    tasa=round(tasa,1)
    return tasa


# Aplicar funcion de calculo de tasa de desercion escoalr
data_Escolar["Tasa_Desercion_Escolar"]=data_Escolar.apply(lambda row: tasa_abandono(row["Estudiantes_Abandono"],row["Total_Estudiantes"]),axis=1)

# Crear un nuevo dataframe , solo con la columna año,provincia y tasa de desercion escolar 
Desercion_Escolar=data_Escolar[["Año","Provincia","Tasa_Desercion_Escolar"]]



,Año,Provincia,Tasa_Desercion_Escolar
0,2009,AZUAY,4.4
1,2009,BOLIVAR,4.1
2,2009,CARCHI,4.9
3,2009,CAÑAR,6.7
4,2009,CHIMBORAZO,4.4
...,...,...,...
364,2023,SANTA ELENA,1.1
365,2023,SANTO DOMINGO DE LOS TSACHILAS,1.9
366,2023,SUCUMBIOS,3.6
367,2023,TUNGURAHUA,2.0


#### Creacion de dataframe con las variables necesarias para el analisis 

##### Como paso previo a la fusion de los dataframes , es necesario unificar el nombre de las provincias 

In [65]:
# Creo un diccionario para remplazar en los dataframes los valores disitintos , las diferencias encontradas solo son tildes y una provincia con abreviatura diferente 

mapeo={"BOLIVAR": "BOLÍVAR",
    "LOS RIOS": "LOS RÍOS",
    "MANABI": "MANABÍ",
    "SUCUMBIOS": "SUCUMBÍOS",
    "SANTO DOMINGO": "SANTO DOMINGO DE LOS TSÁCHILAS",
    "STO DGO DE LOS TSACHILAS": "SANTO DOMINGO DE LOS TSÁCHILAS",
    "SANTO DOMINGO DE LOS TSACHILAS": "SANTO DOMINGO DE LOS TSÁCHILAS",
    "Zona No Delimitada": "ZONA NO DELIMITADA",
    "GALAPAGOS": "GALÁPAGOS"
    
}

# Aplico el reemplazo en los dataframes

Desercion_Escolar.loc[:,"Provincia"]=Desercion_Escolar["Provincia"].replace(mapeo)
df_Pobreza["Provincia"]=df_Pobreza["Provincia"].replace(mapeo)
df_Homicidios["Provincia"]=df_Homicidios["Provincia"].replace(mapeo)


#Fusiono los dataframes
df_final= Desercion_Escolar.merge(df_Homicidios, on=["Año","Provincia"],how= "left")\
    .merge(df_Pobreza, on=["Año","Provincia"], how="left")


#Escogo las columnas y los años necesarios para el analisis 
df_final=df_final[["Año","Provincia","Tasa_Desercion_Escolar","Tasa_Homicidios","Pobreza extrema por ingresos","Pobreza por ingresos","Tasa de pobreza multidimensional"]]
df_final=df_final[df_final["Año"]>=2018].reset_index(drop=True)


#Redondeo las tasas a dos decimales 
df_final["Pobreza extrema por ingresos"] =df_final["Pobreza extrema por ingresos"].round(2)
df_final["Pobreza por ingresos"] =df_final["Pobreza por ingresos"].round(2)
df_final["Tasa de pobreza multidimensional"] =df_final["Tasa de pobreza multidimensional"].round(2)


##Excluyo a Galapagos del analisis dado que existen valores faltantes y sus tasas son muy pequeñas 
df_final=df_final[df_final["Provincia"]!= "GALÁPAGOS"]


df_final

df_final.to_csv("Data_Procesada\\Data_Final.csv",index=False)
